In [1]:
import os

import tensorflow as tf
from sklearn.preprocessing import MultiLabelBinarizer
import numpy as np

from utils import load_data_to_mem, augmentation, show_images


In [2]:
train_path = os.path.join(".", "data", "train")
valid_path = os.path.join(".", "data", "val")
test_path = os.path.join(".", "data", "test")

classes = ["I", "II", "III", "IV", "V", "VI", "VII", "VIII"]
class_mapping = {"I": 0, "II": 1, "III": 2, "IV": 3,
                 "V": 4, "VI": 5, "VII": 6, "VIII":7}
n_classes = len(classes)

def class_to_number(n):
    return class_mapping[n]

img_rows, img_cols, n_channels = 64, 64, 1


In [15]:
X_train, y_train = load_data_to_mem(train_path, classes)
X_valid, y_valid = load_data_to_mem(valid_path, classes)
X_test, y_test = load_data_to_mem(test_path, classes)

X_train, y_train = augmentation(X_train, y_train, n_transform=60)


In [16]:
X_train = np.array(X_train, dtype=np.float32)
X_valid = np.array(X_valid, dtype=np.float32)
X_test = np.array(X_test, dtype=np.float32)
y_train = np.array(y_train)
y_valid = np.array(y_valid)
y_test = np.array(y_test)

# expand dimensions for CNN
X_train = np.expand_dims(X_train, axis=3) 
X_valid = np.expand_dims(X_valid, axis=3)
X_test = np.expand_dims(X_test, axis=3)

y_train = np.array(list(map(class_to_number, y_train)))
y_valid = np.array(list(map(class_to_number, y_valid)))
y_test = np.array(list(map(class_to_number, y_test)))


In [17]:
print("Check shapes: ")
print("X_train =", X_train.shape, " Y_train =", len(y_train))
print("X_valid =", X_valid.shape, " Y_valid =", len(y_valid))
print("X_test =", X_test.shape, " Y_test =", len(y_test))


Check shapes: 
X_train = (60390, 64, 64, 1)  Y_train = 60390
X_valid = (328, 64, 64, 1)  Y_valid = 328
X_test = (338, 64, 64, 1)  Y_test = 338


In [18]:
n_epochs = 15
batch_size = 64


In [19]:
def reset_graph(seed=42):
    tf.reset_default_graph()
    tf.set_random_seed(seed)
    np.random.seed(seed)

#parameters of convolutional layer
conv1_fmaps = 32
conv1_ksize = 5
conv1_stride = 1
conv1_pad = "SAME"

conv2_fmaps = 64
conv2_ksize = 5
conv2_stride = 2
conv2_pad = "SAME"

conv3_fmaps = 64
conv3_ksize = 5
conv3_stride = 2
conv3_pad = "SAME"

#parameters of pooling layer
pool2_fmaps = conv2_fmaps
#parameters of fully connected network and outputs
n_dense1 = 256
n_outputs = 8

reset_graph()

X = tf.placeholder(tf.float32, shape=[None, img_rows, img_cols, n_channels], name = "X")
y = tf.placeholder(tf.int32, shape = [None], name = "y")
    
conv1 = tf.layers.conv2d(X, filters=conv1_fmaps, kernel_size = conv1_ksize,
                         strides = conv1_stride, padding=conv1_pad,
                         activation = tf.nn.relu, name="conv_1")
pool1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                       padding="VALID", name="pool_1")

conv2 = tf.layers.conv2d(pool1, filters=conv2_fmaps, kernel_size=conv2_ksize,
                         strides=conv2_stride, padding=conv2_pad,
                         activation=tf.nn.relu, name="conv_2")
pool2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],
                       padding="VALID", name="pool_2")
    
conv3 = tf.layers.conv2d(pool2, filters=conv3_fmaps, kernel_size=conv3_ksize,
                         strides=conv3_stride, padding=conv3_pad,
                         activation=tf.nn.relu, name="conv_3")
pool3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1],  strides=[1, 2, 2, 1],
                       padding="VALID", name="pool_3")

flat1 = tf.layers.flatten(pool3, name="flatten_1")
dense1 = tf.layers.dense(flat1, n_dense1, activation=tf.nn.relu,
                          name = "dense_1")

with tf.name_scope("output"):
    logits = tf.layers.dense(dense1, n_outputs, name = "output")
    Y_proba = tf.nn.softmax(logits, name="Y_proba")

with tf.name_scope("train"):
    xentropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=y)
    loss = tf.reduce_mean(xentropy)
    optimizer = tf.train.AdamOptimizer()
    training_op = optimizer.minimize(loss)

with tf.name_scope("eval"):
    correct = tf.nn.in_top_k(logits, y, 1)
    accuracy = tf.reduce_mean(tf.cast(correct, tf.float32))

with tf.name_scope("init_and_save"):
    init = tf.global_variables_initializer()
    saver = tf.train.Saver()


### Training CNN. 

Skip next 2 cells if you want to test saved model. 

Training CNN has taken about 30 min with nVidia 940mx GPU.

In [20]:
best_val_score = -9999999
early_stop_patience = 5
last_score = 0


In [21]:
%%time

with tf.Session() as sess:
    init.run()
    for epoch in range(n_epochs):
        for iteration in range(X_train.shape[0] // batch_size):
            X_batch = X_train[iteration * batch_size: iteration * batch_size + batch_size]
            y_batch = y_train[iteration * batch_size: iteration * batch_size + batch_size]
            sess.run(training_op, feed_dict={X: X_batch, y: y_batch})
        acc_train = accuracy.eval(feed_dict={X: X_batch, y: y_batch})
        acc_valid = accuracy.eval(feed_dict={X: X_valid, y: y_valid})
        print("Epoch:", epoch+1, "Train accuracy:", acc_train, "Val accuracy:", acc_valid)
        # save only best model
        if acc_valid > best_val_score:
            best_val_score = acc_valid
            save_path = saver.save(sess, "./saved_models/model1.3_tf")
            print("Model saved.")
        # early stopping
        if acc_valid < last_score:
            early_stop_patience -= 1
        if early_stop_patience <= 0:
            break  


Epoch: 1 Train accuracy: 0.453125 Val accuracy: 0.3628049
Model saved.
Epoch: 2 Train accuracy: 0.6875 Val accuracy: 0.6097561
Model saved.
Epoch: 3 Train accuracy: 0.78125 Val accuracy: 0.6707317
Model saved.
Epoch: 4 Train accuracy: 0.765625 Val accuracy: 0.7012195
Model saved.
Epoch: 5 Train accuracy: 0.796875 Val accuracy: 0.69512194
Epoch: 6 Train accuracy: 0.828125 Val accuracy: 0.72865856
Model saved.
Epoch: 7 Train accuracy: 0.765625 Val accuracy: 0.7134146
Epoch: 8 Train accuracy: 0.890625 Val accuracy: 0.722561
Epoch: 9 Train accuracy: 0.890625 Val accuracy: 0.7530488
Model saved.
Epoch: 10 Train accuracy: 0.875 Val accuracy: 0.76829267
Model saved.
Epoch: 11 Train accuracy: 0.984375 Val accuracy: 0.72865856
Epoch: 12 Train accuracy: 0.953125 Val accuracy: 0.75
Epoch: 13 Train accuracy: 0.96875 Val accuracy: 0.7652439
Epoch: 14 Train accuracy: 0.9375 Val accuracy: 0.7743902
Model saved.
Epoch: 15 Train accuracy: 0.96875 Val accuracy: 0.7835366
Model saved.
Wall time: 16min 58

### Evaluate saved model

In [22]:
def evaluate(X_test, y_test):
    with tf.Session() as sess:
        init = tf.global_variables_initializer()
        sess.run(init) 
        saver = tf.train.import_meta_graph('saved_models/model1.3_tf.meta')
        saver.restore(sess, tf.train.latest_checkpoint('./saved_models/'))
        pred = sess.run([accuracy, loss], feed_dict={X: X_test, y: y_test})
        print("Test accuracy =", pred[0])
        print("Test loss =", pred[1])


In [23]:
evaluate(X_test, y_test)


INFO:tensorflow:Restoring parameters from ./saved_models/model1.3_tf
Test accuracy = 0.75147927
Test loss = 1.1124166
